# Variants

In [9]:
type fruits = Banana | Apple | Avocado ;;
let eat f = match f with Avocado -> "yeah!" | _ -> "nah!" ;; 
eat Avocado ;;

type fruits = Banana | Apple | Avocado


val eat : fruits -> string = <fun>


- : string = "yeah!"


Variants with multiple arguments look an awful lot like tuples. But they are not. In `Pair (10, "hello")` below, `(10, "hello")` is not a 2-tuple.<br/>
The differences between a multi-argument variant and a variant containing a tuple are mostly about performance. A multi-argument variant is a single allocated block in memory, while a variant containing a tuple requires an extra heap-allocated block for the tuple.

In [10]:
type data = 
    | Int of int
    | String of string
    | Pair of int * string ;;
open Printf ;;
let show = function 
            | Int x -> sprintf "Int %d" x 
            | String x -> sprintf "String %s" x
            | Pair (x, s) -> sprintf "Int %d, String %s" x s ;;
show (Int 10) ;;
show (String "hello") ;;
show (Pair(10, "hello")) ;;

type data = Int of int | String of string | Pair of int * string


val show : data -> string = <fun>


- : string = "Int 10"


- : string = "String hello"


- : string = "Int 10, String hello"


The subtle difference between variant multiple arguments and tuple is the later can be grabbed as a whole.

In [23]:
type point_m = Point of float * float ;;
let pm = Point (1.1, 2.2) ;;

(* this won't work *)
(* let (Point x) = pm ;;  *)

let (Point (x,y)) = pm ;;

type point_t = TupledPoint of (float * float) ;;
let pt = TupledPoint (4.4, 5.5) ;;

let (TupledPoint x) = pt ;;
x ;;
let (TupledPoint (x,y)) = pt ;;

type point_m = Point of float * float


val pm : point_m = Point (1.1, 2.2)


val x : float = 1.1
val y : float = 2.2


type point_t = TupledPoint of (float * float)


val pt : point_t = TupledPoint (4.4, 5.5)


val x : float * float = (4.4, 5.5)


- : float * float = (4.4, 5.5)


val x : float = 4.4
val y : float = 5.5


Variants with inline records are both more concise and more efficient than having variants containing references to free-standing record types, because they don’t require a separate allocated object for the contents of the variant.
<br/>
The main downside is the obvious one, which is that an inline record can’t be treated as its own free-standing object.
<br/>
https://dev.realworldocaml.org/variants.html#variants-tuples-and-parens

In [33]:
type circle = { radius : float; centerX : float; centerY : float } ;;

type shape = 
    | Square of float
    | Rectangle of { width : float; height : float }
    | Circle of circle

let check_shape s =
    match s with
    | Square x -> Printf.sprintf "Square of %F" x
    | Rectangle r -> Printf.sprintf "Rect with width: %F, height: %F" r.width r.height
    | Circle c -> Printf.sprintf "Circle at %F,%F with radius %F" c.centerX c.centerY c.radius ;;

check_shape (Rectangle {width = 100.; height = 200.}) ;;
check_shape (Circle {radius = 10.; centerX = 10.; centerY=10.}) ;;

(* but this won't work *)
let get_shape s = 
    match s with
    | Circle c -> c  (* this works cause circle is a free-standing record type  *)
    | Rectangle r -> r (*  this won't work cause r is inline record *)
    | Square x -> x ;;

type circle = { radius : float; centerX : float; centerY : float; }


type shape =
    Square of float
  | Rectangle of { width : float; height : float; }
  | Circle of circle


val check_shape : shape -> string = <fun>


- : string = "Rect with width: 100., height: 200."


- : string = "Circle at 10.,10. with radius 10."


error: compile_error

##### 🟢 &nbsp; &nbsp; Variants and Recursive Data Structures

In [34]:
type 'a expr =
 | Base of 'a
 | Const of bool
 | And of 'a expr list
 | Or of 'a expr list
 | Not of 'a expr ;;

type 'a expr =
    Base of 'a
  | Const of bool
  | And of 'a expr list
  | Or of 'a expr list
  | Not of 'a expr
